# scrapy
Scrapy nos permite manejar la request de manera paralela asincrónica. 
En vez de como le hacíamos con requests y BS de: 
1.	enviar request
2.	procesar la respuesta 
3.	hacer la próxima request 
con scrapy enviaremos y procesaremos de forma paralela. 
Otras ventajas: 
-	ya no debemos parsear con BS ya que podemos usar Xpath directamente 
-	limitar la cantidad de requests en paralelo 
-	setear demoras que nos indica el robots.tx

In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess

# practica analisis en paralelo 
crearemos un scrapper capaz de obtener el html de 2 paginas de todas las secciones del diario

In [10]:
# creamos una herencia del objeto scrapy spider
class Spider12(scrapy.Spider):
    name = 'spider12'
    # dominios a scrapear
    allowed_domains = ['pagina12.com.ar']
    # formato de archivo de salida
    custom_settings = { 'FEED_FORMAT':'json',   
                        'FEED_URI': 'resultados.json',
                        'FEED_EXPORT_ENCODING': 'utf-8',
                        'DEPTH_LIMIT' : 2}

    # URLS a scrapear. son todas las secciones del diario 
    starts_urls = [ 'https://www.pagina12.com.ar/secciones/el-pais',
                    'https://www.pagina12.com.ar/secciones/economia',
                    'https://www.pagina12.com.ar/secciones/sociedad',
                    'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                    'https://www.pagina12.com.ar/secciones/el-mundo',
                    'https://www.pagina12.com.ar/secciones/deportes',
                    'https://www.pagina12.com.ar/secciones/contratapa',
                    'https://www.pagina12.com.ar/secciones/audiovisuales']
    
    # creamos un metodo obtener los links a cada nota 
    def parse(self, response): # response es el equivalente de hacer un requests.get()
        # Listado de  links notas
        links_notas = response.xpath("//h4[@class='title-list']/a/@href").getall() # faltan h3,h2
        for nota in links_notas: 
            # canalizar la respuesta hacia el metodo parse_nota
            # yield == return, solo que direcciona el resultado a otro metodo
            # for url in articles_title:
            nota = 'https://www.pagina12.com.ar' + nota
            yield response.follow(nota, callback=self.parse_nota)
        # creamos accedemos al boton de siguiente pagina
        next_page = 'https://www.pagina12.com.ar{}'.format(response.xpath('//a[@class="next"]/@href').get())
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)
    # creamos un metodo para analizar contenido de cada nota 
    def parse_nota(self, response):
        title = response.xpath('//h1/text()').get()
        summary = response.xpath('//h2/text()').get() 
        yield{"url": response.url,
                "titulo": title,
                "resumen": summary}

In [14]:
"""process = CrawlerProcess()
process.crawl(Spider12)
process.start()"""


'process = CrawlerProcess()\nprocess.crawl(Spider12)\nprocess.start()'

# proxys
Como ya sabes un scrapper es un programa que automatice consultas(requests) a paginas web. Todas las consultas salen con desde un sola PC y por lo tanto tienen la misma IP, el servidor web detectara que hay un bot y te bloqueara cuando detecta:
-	Alta frecuencia de peticiones desde una sola IP
Es por eso que necesitamos enmascarar las peticiones usando un proxy. El proxy es un intermediario entre nosostros y el servidor final, asi el servidor detectara la IP del proxy y no la nuestra. Podemos configurar al programa para que use varias proxys y engañe al servidor.

Para saber tu IP publica con la que te identificas en la web  puedes usar : https://www.cualesmiip.com/


## encontrar tu IP publica
tambien podemos esta pagina que nos regresa directamente la IP, la usaremos para ver que pasa si ne nuestra request asignamos otra IP de un proxy.

In [16]:
import requests

In [17]:
print(requests.get('https://ident.me/').text)

2023-01-31 15:46:55 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): ident.me:443
2023-01-31 15:46:57 [urllib3.connectionpool] DEBUG: https://ident.me:443 "GET / HTTP/1.1" 200 14


200.68.173.193


In [22]:
def get_my_ip(url='https://ident.me/', proxies=None):   
    # Hacemos la request al sitio
    try:
        resp = requests.get(url, proxies=proxies).text
    except Exception as e:
        print('Error haciendo la request.', e)
        return None
    return resp

In [21]:
get_my_ip()

2023-01-31 16:24:22 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): ident.me:443
2023-01-31 16:24:22 [urllib3.connectionpool] DEBUG: https://ident.me:443 "GET / HTTP/1.1" 200 14


200.68.173.193


'200.68.173.193'

## configurar un proxy
para configurar un proxy simplemente tenemos que crear un URL con:
- protocolo de transferencia: http, https
- IP publico del proxy: ejemplo 135.181.14.45
- puerto del proxy: ejemplo 5959
al final construyes un diccionario especificando un proxy http y otro https
```python
proxy_dict = {'http':'http://135.181.14.45:5959',
              'https':'http://135.181.14.45:5959'}
# ahora lo podemos pasar a la peticion
r = requests.get(url, proxies=proxies)
```
arriba ocupamos la misma IP ya que esta aplicaba para el protocolo https y por lo tanto ya incluye http
estos datos los podemos obtener de un pagina como: 
https://free-proxy-list.net/

In [23]:
proxy_dict = {'http':'http://135.181.14.45:5959',
            'https':'http://135.181.14.45:5959'}
get_my_ip(proxies=proxy_dict)
# ahora la IP con la que nos identificamos en internet es la del proxy y no la nuestra

2023-01-31 16:33:47 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): ident.me:443
2023-01-31 16:33:49 [urllib3.connectionpool] DEBUG: https://ident.me:443 "GET / HTTP/1.1" 200 13


'2.199.247.184'

## proxys socks 
es lo mismo que arriba solo cambiamos la version de sock por el protocolo htpp pero ahora tienes que entrar a la pagina a consultar algun proxy sock https://www.socks-proxy.net/
```python
proxy_dict_socks = {'http':'socks4://138.97.116.171:50659',
                    'https':'socks4://138.97.116.171:50659'}
```
recuerda que socks trabajan con protocolo TCP. Mas vijos pero lo podemos utilizar para cualquier tipo de tráfico ya sean páginas web, programas, torrents, etc.

In [24]:
proxy_dict_socks = {'http':'socks4://36.67.27.189:49524',
                    'https':'socks4://36.67.27.189:49524'}
get_my_ip(proxies=proxy_dict_socks)

2023-01-31 16:42:00 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): ident.me:443
2023-01-31 16:42:05 [urllib3.connectionpool] DEBUG: https://ident.me:443 "GET / HTTP/1.1" 200 12


'36.67.27.189'

# pytesseract
En esta clase veremos algunos ejemplos de utilización de Tesseract-OCR, una biblioteca creada por HP (y actualmente mantenida por Google) para realizar reconocimiento óptico de caracteres.

In [25]:
import tesserocr # Para hacer OCR
import numpy as np # Para hacer manipulación básica de imágenes
import matplotlib.pyplot as plt # Para visualizar imágenes
from PIL import Image # Para cambiar el formato de archivos
%matplotlib inline

ModuleNotFoundError: No module named 'tesserocr'

In [ ]:
texto_largo = plt.imread('texto_largo.png')
plt.figure(figsize=(15,5))
plt.imshow(texto_largo)
plt.axis(False)
plt.show()

In [ ]:
texto_ocr = tesserocr.file_to_text('texto_largo.png', lang='spa')
print(texto_ocr)

ahora intentemos con una imagen mas dificial la cual tiene 4 canales, las letras son color blanco y los bordes no estan bien definidos

In [ ]:
img = plt.imread('imagen de prueba.png')
plt.imshow(img)
texto_ocr = tesserocr.file_to_text('imagen de prueba.png', lang='spa')
print(texto_ocr)
# vemos como hay muchas fallas asi que debemos preprocesar las imagenes 

In [ ]:
img_rgb = img[:,:,:3] # eliminamos el canal de transparencia
plt.imshow(img_rgb)

In [ ]:
print(img_rgb[0,0,0]) # invertimos el color. para esto necesitamos saber el rango para hacer la correccion
img_inv = 1 - img_rgb
plt.imshow(img_inv)

In [ ]:
# pasamos de imagen rgb a b/n
img_gr = img_inv.mean(axis=2) # sacamos el promedio de rbg 
plt.imshow(img_gr, cmap='Greys_r' )

In [ ]:
# tesseract no está preparado para trabajar con arrays de numpy. Debemos convertir el formato a Image
img_pil = Image.fromarray(np.uint8(img_gr*255))
print(tesserocr.image_to_text(img_pil, lang='spa'))

Mucho mejor, pero seguimos teniendo un problema ya que este curso es de Web Scraping, no Heb Scraping. Tal vez perdimos demasiada información al pasar la imagen a escala de grises. Intentemos con la imagen invertida en RGB.

In [ ]:
img_pil_inv = Image.fromarray(np.uint8(img_inv*255))
print(tesserocr.image_to_text(img_pil_inv, lang='spa'))